In [1]:
# !pip install opencv-python matplotlib tensorflow-datasets

In [2]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Conv2D, MaxPooling2D, Input, Flatten, Dense
import tensorflow as tf
import tensorflow_datasets as tfds

2024-09-09 14:44:16.754539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 14:44:16.772568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 14:44:16.778076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 14:44:16.792044: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-09 14:44:17.678030: W tensorflow/compiler/tf2

# Set GPU Growth

In [4]:
#avoid out of memory erros by setting GPU memory growth

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


2024-09-09 14:44:18.320664: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-09-09 14:44:18.320694: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: nitro
2024-09-09 14:44:18.320701: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: nitro
2024-09-09 14:44:18.320809: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.183.1
2024-09-09 14:44:18.320830: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 535.183.1
2024-09-09 14:44:18.320836: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 535.183.1


In [5]:
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

## Folder structures

In [6]:
POS_PATH = os.path.join("data", "positive")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor")

In [7]:
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

In [8]:
lfw_dataset, info = tfds.load('lfw', split='train', with_info=True, as_supervised=True)

# Display dataset information
print(info)

tfds.core.DatasetInfo(
    name='lfw',
    full_name='lfw/0.1.0',
    description="""
    Labeled Faces in the Wild: A Database for Studying Face Recognition in
    Unconstrained Environments
    """,
    homepage='http://vis-www.cs.umass.edu/lfw',
    data_dir='/home/andersonc/tensorflow_datasets/lfw/0.1.0',
    file_format=tfrecord,
    download_size=172.20 MiB,
    dataset_size=180.28 MiB,
    features=FeaturesDict({
        'image': Image(shape=(250, 250, 3), dtype=uint8),
        'label': Text(shape=(), dtype=string),
    }),
    supervised_keys=('label', 'image'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=13233, num_shards=2>,
    },
    citation="""@TechReport{LFWTech,
        author = {Gary B. Huang and Manu Ramesh and Tamara Berg and Erik Learned-Miller},
        title = {Labeled Faces in the Wild: A Database for Studying Face Recognition in Unconstrained Environments},
        institution = {University of Massachusetts, Amherst},
     

In [9]:
# Function to save images to a directory
def save_image(image, index, directory):
    # Convert the image tensor to a NumPy array and then to uint8
    image_np = tf.image.convert_image_dtype(image, dtype=tf.uint8).numpy()

    # Encode image in JPEG format
    image_encoded = tf.image.encode_jpeg(image_np)
    
    # Define the path to save the image
    image_path = os.path.join(directory, f'image_{index}.jpg')
    
    # Write the image to the file
    tf.io.write_file(image_path, image_encoded)

# Move all images to the negative directory
for index, (label, image) in enumerate(lfw_dataset):
    save_image(image, index, NEG_PATH)

print(f"All images have been moved to the {NEG_PATH} directory.")

All images have been moved to the data/negative directory.


2024-09-09 14:44:33.144158: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Colect Positive and Anchor Classes

In [10]:
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         print("Can't receive frame (stream end?). Exiting ...")
#         break

#     cv2.imshow('frame', frame)
#     if cv2.waitKey(1) & 0XFF == ord('q'):
#         break

#     if cv2.waitKey(1) & 0XFF == ord('p'):
#         cv2.imwrite(os.path.join(POS_PATH, f"image_{random.randint(0, 1000)}.jpg"), frame)  
    
# cap.release()
# cv2.destroyAllWindows()

In [11]:
#Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        (x, y, w, h) = faces[0] 
        center_x, center_y = x + w // 2, y + h // 2
        
        # Define the dimensions for the centered crop
        half_width = 125  # Half of the target width 250
        half_height = 125  # Half of the target height 250

        
        start_x = max(center_x - half_width, 0)
        start_y = max(center_y - half_height, 0)
        end_x = min(center_x + half_width, frame.shape[1])
        end_y = min(center_y + half_height, frame.shape[0])
        
        
        cropped_frame = frame[start_y:end_y, start_x:end_x]
        
        
        resized_frame = cv2.resize(cropped_frame, (250, 250))
    else:
        
        resized_frame = cv2.resize(frame, (250, 250))
    
    
    cv2.imshow('frame', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('p'):
        image_path = os.path.join(POS_PATH, f"{uuid.uuid1()}.jpg")
        cv2.imwrite(image_path, resized_frame)
        print(f"Saved frame to {image_path}")

    if cv2.waitKey(1) & 0xFF == ord('a'):
        image_path = os.path.join(ANC_PATH, f"{uuid.uuid1()}.jpg")
        cv2.imwrite(image_path, resized_frame)
        print(f"Saved frame to {image_path}")
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    

cap.release()
cv2.destroyAllWindows()

Saved frame to data/anchor/30230f92-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/30a0edf4-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/30bb131e-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/30d3157c-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/30ee6138-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/3120004e-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/313a2bc2-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/315755bc-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/316ff95a-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/318a1f74-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/31c9ec08-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/31e721ba-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/32015cce-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/positive/323e67ae-6ed3-11ef-98bd-dc215cb50b44.jpg
Saved frame to data/po

# Load and process images

In [12]:
anchor = tf.data.Dataset.list_files(ANC_PATH + "/*.jpg").take(30)
positive = tf.data.Dataset.list_files(POS_PATH + "/*.jpg").take(30)
negative = tf.data.Dataset.list_files(NEG_PATH + "/*.jpg").take(30)


In [13]:
dir_test = anchor.as_numpy_iterator().next().decode("utf-8")
print(dir_test)

data/anchor/bef14674-67be-11ef-8b57-dc215cb50b44.jpg


In [14]:
def preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [100, 100])
    img = img / 255.0
    return img

In [15]:
img = preprocess_image(dir_test)

In [ ]:
plt.imshow(img)

In [17]:
# create labelled dataset

class_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))
iterator_labs = class_labels.as_numpy_iterator()
iterator_labs.next()


0.0

In [18]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
dataset = tf.data.Dataset.concatenate(positives, negatives)

In [19]:
samples = dataset.as_numpy_iterator()

In [20]:
samples.next()

(b'data/anchor/c3ca9682-67be-11ef-8b57-dc215cb50b44.jpg',
 b'data/positive/3d15c09e-67c3-11ef-80ac-dc215cb50b44.jpg',
 1.0)

In [21]:
def process_twin(input_img, validatioin_img, label):
    return preprocess_image(input_img), preprocess_image(validatioin_img), label

In [22]:
res = process_twin(*samples.next())

In [23]:
#build dataloader pipeline

dataset = dataset.map(process_twin)
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size=1024)

In [24]:
# Training  partitions
train_data = dataset.take(round(len(dataset)*0.8))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [25]:
# Testing partitions
test_data = dataset.skip(round(len(dataset)*0.8))
test_data = test_data.take(round(len(dataset)*0.2))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)


# Model Engineering

## Embedding Layer

In [26]:
def make_embedding():
    input = Input(shape=(100, 100, 3), name="input_image")
    c1 = Conv2D(64, (10, 10), activation='relu')(input)
    m1 = MaxPooling2D(64, (2, 2), padding='same')(c1)

    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2, 2), padding='same')(c2)

    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2, 2), padding='same')(c3)

    c4 = Conv2D(256, (4, 4), activation='relu')(m3)
    f = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f)
    return Model(inputs=[input], outputs=[d1], name="embedding")

In [27]:
model = make_embedding()

In [28]:
model.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 91, 91, 64)     │        19,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 40, 40, 128)    │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 17, 17, 128)    │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 256)      │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,960,448 (148.62 MB)

 Trainable params: 38,960,448 (148.62 MB)

 Non-trainable params: 0 (0.00 B)

## Distance Layer

In [29]:
# Siamese L1 Distance Class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        input_embedding = tf.convert_to_tensor(input_embedding)
        validation_embedding = tf.convert_to_tensor(validation_embedding)
        return tf.math.abs(input_embedding - validation_embedding)
    

## Siamese Model

In [30]:
input_image = Input(name='input_img', shape=(100,100,3))
validation_image = Input(name='validation_img', shape=(100,100,3))

In [31]:
inp_embedding = model(input_image)
val_embedding = model(validation_image)

In [32]:
print(inp_embedding)

[<KerasTensor shape=(None, 4096), dtype=float32, sparse=False, name=keras_tensor_9>]


In [33]:
siamese_layer = L1Dist()

In [34]:
distances = siamese_layer(inp_embedding, val_embedding)

In [35]:
def siamese_model():
    input = Input(shape=(100, 100, 3), name="input_image")
    validation = Input(shape=(100, 100, 3), name="validation_image")

    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(model(input), model(validation))

    classifier = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[input, validation], outputs=[classifier], name="siamese")

In [36]:
siamese_model = siamese_model()

In [37]:
siamese_model.summary()

Model: "siamese"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_image    │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4096)      │ 38,960,448 │ input_image[0][0… │
│ (Functional)        │                   │            │ validation_image… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_dist_1 (L1Dist)  │ (1, None, 4096)   │          0 │ embedding[2][0],  │
│                     │                   │            │ embedding[3][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (1, None, 1)      │      4,097 │ l1_dist_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

# Training

In [38]:
# Loss and optimizer
binary_cross_loss = tf.losses.BinaryCrossentropy()


In [39]:
opt = tf.keras.optimizers.Adam(0.001)

In [40]:
#checkpoints

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=opt, model=model)

## Train step function

In [41]:
test_batch = train_data.as_numpy_iterator()

In [42]:
batch_1 = test_batch.next()

In [43]:
len(batch_1[0])

16

In [44]:
X = batch_1[:2]

In [45]:
np.array(X).shape

(2, 16, 100, 100, 3)

In [46]:
y = batch_1[2]

In [47]:
y

array([0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
      dtype=float32)

In [48]:
np.array(y).shape

(16,)

In [49]:
y_pred = siamese_model(X, training=True)

In [50]:
y_pred

<tf.Tensor: shape=(1, 16, 1), dtype=float32, numpy=
array([[[0.50637233],
        [0.5008936 ],
        [0.5020103 ],
        [0.50200874],
        [0.5017967 ],
        [0.5000565 ],
        [0.5014882 ],
        [0.5017015 ],
        [0.5014063 ],
        [0.5021161 ],
        [0.5025069 ],
        [0.50204635],
        [0.5022658 ],
        [0.50151914],
        [0.5039177 ],
        [0.5012044 ]]], dtype=float32)>

In [51]:
tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]

        #forward pass
        y_pred = siamese_model(X, training=True)
        y_pred = tf.squeeze(y_pred, axis=[0, 2])
        loss = binary_cross_loss(y, y_pred)
    print(f"Loss: {loss}")

    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    return loss

## training loop

In [52]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
        
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

## train the model

In [53]:
EPOCHS = 50

In [54]:
train(train_data, EPOCHS)


 Epoch 1/50
Loss: 0.6940973997116089
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.6674364805221558
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.732749342918396
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 2/50


2024-09-09 14:45:20.364614: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Loss: 0.7333803772926331
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.6774736046791077
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.6211386322975159
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 3/50
Loss: 0.5929840803146362
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.6237573623657227
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.567034125328064
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 4/50


2024-09-09 14:45:48.719184: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Loss: 0.5013681650161743
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.7287524938583374
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.48856231570243835
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 5/50
Loss: 0.5199129581451416
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.4955675005912781
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.4547836184501648
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 6/50
Loss: 0.45399072766304016
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.35049742460250854
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.36602461338043213
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step

 Epoch 7/50
Loss: 0.34817177057266235
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.2986963093280792
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.38152793049812317
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 8/50


...

Loss: 0.1783258616924286
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.3181908130645752
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.15637484192848206
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 9/50
Loss: 0.19016101956367493
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.1354123204946518
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.08228069543838501
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 10/50
Loss: 0.08792419731616974
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.08894182741641998
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 5s/stepLoss: 0.14480000734329224
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step

 Epoch 11/50
Loss: 0.05012156069278717
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.1681365817785263
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.12133382260799408
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 12/50
Loss: 0.11855576187372208
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.12374909222126007
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.036481793969869614
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 13/50
Loss: 0.0

2024-09-09 14:48:32.365146: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Loss: 0.024458950385451317
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.03476733714342117
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.016237769275903702
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 17/50
Loss: 0.0061633652076125145
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.003832070389762521
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.006648017093539238
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 18/50
Loss: 0.0028661072719842196
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.0008253723499365151
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.013624824583530426
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 19/50
Loss: 0.001538627315312624
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 0.00442373426631093
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.0017847028793767095
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 20/50
Loss: 0.008391511626541615
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 0.017552653327584267
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 0.0017918975790962577
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step



2024-09-09 14:52:10.375982: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Loss: 3.382433351362124e-05
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 1.6428877643193118e-05
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 6.37368721072562e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 33/50
Loss: 5.9162739489693195e-05
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 6.0842085076728836e-05
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 3.838106931652874e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 34/50
Loss: 3.9404745621141046e-05
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/stepLoss: 1.557219547976274e-05
2/3 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepLoss: 6.469025538535789e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step

 Epoch 35/50
Loss: 6.18729172856547e-05
1/3 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/stepLoss: 3.357636887812987e-05
2/3 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step Loss: 6.3144307205220684e-06
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 5s/step

 Epoch 36/50
Loss: 2.6767407689476386e-05
1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/stepLoss: 4.8370678996434435e-05
2/3 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/stepLoss: 7.141476089600474e-06
3/3 ━━━━━━━━━━━━━━━

## Evaluations

In [55]:
from tensorflow.keras.metrics import Precision, Recall

In [56]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [57]:
predictions = siamese_model.predict([test_input, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step


In [66]:
type(predictions)
print(predictions.shape)

(1, 12, 1)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [71]:
# [1 if prediction > 0.5 else 0 for prediction in predictions.numpy()]
binary_predictions = tf.where(predictions > 0.5, 1, 0)

In [72]:
y_true

array([1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1.], dtype=float32)

In [75]:
m = Recall()
m.update_state(y_true, binary_predictions)
m.result().numpy()

1.0

In [76]:
n = Precision()
n.update_state(y_true, binary_predictions)
n.result().numpy()

1.0

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(1,2,)
plt.imshow(test_input[3])
plt.title("Input Image")
plt.subplot(1,2,2)
plt.imshow(test_val[3])
plt.title("Validation Image")
plt.show()

In [82]:
siamese_model.save('siamese_model.h5')

In [85]:
model = tf.keras.models.load_model('siamese_model.h5', custom_objects={'L1Dist': L1Dist, 'BinaryCrossEntropy': tf.losses.BinaryCrossentropy()})